In [ ]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

# Загрузка данных
file_path = '/content/drive/MyDrive/Laptop_price.csv'  # Замените на путь к файлу
try:
    df = pd.read_csv(file_path)  # Читаем CSV-файл и загружаем его в DataFrame
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")

# 1. Проверка CSV на уязвимости (CSV Injection)
# Дополнительно проверим на пробелы перед опасными символами и SQL-комментарии
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')
    comment_pattern = '--'  # Комментарий в SQL-запросах

    for col in df.select_dtypes(include=['object']).columns:
        # Проверка на начинающиеся с пробелом перед символом или самими символами
        starts_with_dangerous = df[col].astype(str).apply(
            lambda x: any(x.strip().startswith(char) for char in dangerous_chars)
        ).any()

        # Проверка на SQL-комментарии
        contains_sql_comment = df[col].astype(str).apply(lambda x: comment_pattern in x).any()

        if starts_with_dangerous or contains_sql_comment:
            print(f"Обнаружены потенциальные CSV-инъекции или SQL-комментарии в столбце {col}!")
        else:
            print(f"Столбец {col} безопасен.")

check_csv_injection(df)

# 2. Фильтрация данных от SQL-инъекций и XSS-атак
# SQL-инъекции используются для взлома баз данных, а XSS – для атак на пользователей
# Добавим проверку на SQL-комментарии
def clean_input(value):
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION", "--"]  # Добавлены SQL-комментарии
    xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror=.*']

    for keyword in sql_keywords:
        if keyword.lower() in value.lower():
            return "[BLOCKED]"

    for pattern in xss_patterns:
        if re.search(pattern, value):
            return "[BLOCKED]"

    return value

df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")

# 3. Хеширование столбца с ценами (SHA-256)
# Хеширование используется для защиты конфиденциальных данных, таких как пароли
# В данном случае мы хешируем цену ноутбука, чтобы скрыть оригинальное значение
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

df['Price_Hashed'] = df['Price'].apply(hash_price)
print("Столбец с хешированными ценами добавлен.")

# 4. Шифрование данных (например, цены ноутбука)
# Шифрование позволяет скрыть данные, но при этом их можно расшифровать при наличии ключа
# Генерируем ключ и шифруем цену ноутбука
key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_price(price):
    return cipher.encrypt(str(price).encode()).decode()

def decrypt_price(encrypted_price):
    return cipher.decrypt(encrypted_price.encode()).decode()

df['Price_Encrypted'] = df['Price'].apply(encrypt_price)
print("Столбец с зашифрованными ценами добавлен.")

# 5. Шифрование столбца RAM
# Добавление новой колонки с зашифрованной RAM
def encrypt_ram(ram):
    return cipher.encrypt(str(ram).encode()).decode()

df['RAM_Encrypted'] = df['RAM'].apply(encrypt_ram)
print("Столбец с зашифрованной RAM добавлен.")

# 6. Функционал расшифровки RAM
# Декодируем первые 5 записей RAM
def decrypt_ram(encrypted_ram):
    return cipher.decrypt(encrypted_ram.encode()).decode()

first_five_decrypted_ram = df['RAM_Encrypted'].head(5).apply(decrypt_ram)
for idx, dec_ram in first_five_decrypted_ram.items():
    print(f"Расшифрованная RAM №{idx}: {dec_ram}")

# Сохранение обработанных данных
output_path = "/content/drive/MyDrive/Laptop_price_secure.csv"
df.to_csv(output_path, index=False)
print(f"Обработанный файл сохранён: {output_path}")